In [2]:
!pip install neo4j

In [3]:
from datetime import datetime

from neo4j import GraphDatabase, basic_auth

In [4]:
URL = "bolt://localhost:7687"
USERNAME = "Daniel_Hurtado"
PASSWORD = "123456"
DATABASE = "fourpaws"

In [5]:
driver = GraphDatabase.driver(URL, auth = basic_auth(USERNAME, PASSWORD))

In [6]:
db = driver.session(database = DATABASE)

### Creating a basic graph

<img src="imgs/fig1.png">

In [8]:
def crearPropietarioyMascota (nameP, typeM, nameM):
    summary = db.write_transaction(lambda tx: tx.run("CREATE (:Person:Owner {name: '" + nameP + "'})-[:OWNS]->(:Pet:" + typeM + " {name:'" + nameM + "'})").consume())

In [34]:
crearPropietarioyMascota("Daniel", "Cat", "Lola")

In [11]:
def cargarFotoyAsociarla(nameM, urlIm): 
    summary = db.write_transaction(lambda tx: tx.run("MATCH (d:Pet {name: '"+ nameM +"'}) CREATE (i:Picture {createdAt:  $createdAt}) CREATE (d)-[:APPEARS_IN]->(i)", {"createdAt": urlIm}).consume())

In [23]:
cargarFotoyAsociarla("Mailo", "https://www.muyinteresante.es/mascotas/fotos")

In [50]:
def reaccionarFoto(urlI, namePersona, typeReaction):
    db.write_transaction(lambda tx: tx.run("MATCH (i:Picture {createdAt: '" + urlI +"' }) MATCH (c:Person {name: '" + namePersona + "'}) CREATE (c)-[:"+ typeReaction +"]->(i)").consume())

In [74]:
reaccionarFoto("https://www.muyinteresante.es/mascotas/fotos/impresionantes-curiosidades-sobre-perros-que-fuerte", "Daniel", "Like")

In [41]:
def asociarFoto(urlI, nameMascota, typeMascota):
    db.write_transaction(lambda tx: tx.run("MATCH (i:Picture {createdAt: '" + urlI +"' }) MATCH (c:Pet: "+ typeMascota +" {name: '" + nameMascota + "'}) CREATE (c)-[:APPEARS_IN]->(i)").consume())

In [47]:
asociarFoto("https://www.muyinteresante.es/mascotas/fotos", "Zeus", "Dog")

### Counting nodes by type

In [43]:
result = db.read_transaction(lambda tx: tx.run("MATCH (:Dog) RETURN count(*) AS count").single())

In [44]:
result["count"]

1

### Counting number of outbound relationships from a node

In [48]:
# For a specific node
result = db.read_transaction(lambda tx: tx.run("MATCH (p:Pet) WHERE ID(p) = $node RETURN size((p)-->()) AS count", {"node": 4}).single())

In [49]:
result["count"]

3

In [88]:
# For a group of nodes
result = db.read_transaction(lambda tx: list(tx.run("MATCH (p:Person)-[r:Like]->(i:Picture) RETURN i.createdAt AS url, COUNT(r) AS num_likes")))

In [89]:
result.sort()
for r in result:
    print(r["url"] + " => " + str(r["num_likes"]))

http://c.files.bbci.co.uk/48DD/production/_107435681_perro1.jpg => 1
https://www.muyinteresante.es/mascotas/fotos => 3
https://www.muyinteresante.es/mascotas/fotos/impresionantes-curiosidades-sobre-perros-que-fuerte => 2


In [ ]:
db.close()